In [1]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import scipy
import sklearn

movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

movie_texts = movies['overview'].apply(str).to_list()
movie_titles = movies['title'].apply(str).to_list()
text_corpus = []
for i in range(len(movie_texts)):
    if movie_titles[i] != 'nan' and movie_texts[i] != 'nan':
            text = movie_titles[i] + " " + movie_texts[i]
            text_corpus.append(text)

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
import nltk
words = set(nltk.corpus.words.words())

def remove_non_english(sent):
    s = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
    return s

In [15]:
test = df[df['title'] == 'Toy Story']
print(test['overview'][0])
remove_non_english(test['overview'][0])

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


"Led by Woody , ' s live happily in his room until ' s birthday Buzz onto the scene . Afraid of losing his place in ' s heart , Woody against Buzz . But when separate Buzz and Woody from their owner , the duo eventually to put aside their ."

In [3]:
df = movies.filter(['id','title','overview'], axis=1)

In [4]:
# Create new movies df, with only important columns
df = movies.filter(['id','title','overview'], axis=1)
print('Before: ' + str(len(df)))
df = df.dropna()
# check movies ids, convert to int, drop if not possible
for id in df['id'].to_list():
    try:
        id = int(id[0])
    except ValueError:
        df = df[df['id'] != id]

for title in df['title'].to_list():
    try:
        title = str(title)
        if title == 'nan' or title == '' or title == ' ':
            df = df[df['title'] != title]
        elif len(remove_non_english(title)) < len(title)/4:
            df = df[df['title'] != title]
    except ValueError:
        df = df[df['title'] != title]

for overview in df['overview'].to_list():
    try:
        overview = str(overview)
        if overview == 'nan' or overview == '' or overview == ' ' or overview == '...':
            df = df[df['overview'] != overview]
        elif len(remove_non_english(overview)) < len(overview)/2:
            df = df[df['overview'] != overview]
    except ValueError:
        df = df[df['overview'] != overview]

df = df.reset_index(drop=True)

print('After: ' + str(len(df)))

Before: 45466
After: 37713


In [ ]:
22975
39063 title /10
33165 title /2

In [5]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer("paraphrase-distilroberta-base-v1")

#Our sentences we like to encode
#sentences = ['This framework generates embeddings for each input sentence',
#    'Sentences are passed as a list of string.', 
#    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
#sentence_embeddings = model.encode(sentences)


In [6]:
import pickle

In [12]:
bert_embeddings = []
with open('bert_embeddings_data(paraphrase-distilroberta-base-v1).pickle', 'rb') as f:
    bert_embeddings = pickle.load(f)

In [13]:
print(len(bert_embeddings))
bert_embeddings[0]

37713


array([ 1.08312011e-01,  6.03061974e-01,  2.08922401e-01, -9.10238624e-02,
        4.09882873e-01, -1.46170393e-01, -4.06475626e-02,  2.63245732e-01,
        4.81473655e-01, -6.95433468e-02,  2.06151083e-01,  4.80479062e-01,
        1.56298280e-02,  2.24300787e-01,  2.30348438e-01,  2.57539749e-01,
       -2.99162362e-02,  1.12278134e-01, -6.82995319e-02,  3.44873577e-01,
       -7.87067264e-02, -4.11573857e-01,  1.67148829e-01,  1.44687235e-01,
       -2.53175735e-01,  2.53225565e-01,  1.78524047e-01,  4.47399318e-01,
       -4.96918708e-02,  2.88475186e-01,  9.36294124e-02, -1.29827276e-01,
       -1.75479889e-01, -2.66902405e-03,  9.78612602e-02, -2.41087037e-04,
       -4.07178611e-01,  1.69955999e-01,  5.49014986e-01, -2.29082718e-01,
       -1.32328883e-01,  2.75524914e-01,  2.11583033e-01, -7.61252159e-05,
        1.70351297e-01,  1.73601851e-01, -3.34479451e-01, -6.16944194e-01,
       -2.38201082e-01,  9.78883076e-03,  1.68363377e-01,  1.65228739e-01,
       -1.42783567e-01,  

In [26]:
sim = cosine_similarity(bert_embeddings, bert_embeddings)

In [19]:
sim = np.sort(sim)

In [21]:
sim[0][37700:]

array([0.46505326, 0.46551484, 0.4665162 , 0.47398788, 0.47433412,
       0.4898186 , 0.49345905, 0.49389705, 0.49547583, 0.50280875,
       0.57592094, 0.5922534 , 1.        ], dtype=float32)

In [43]:
s = sim[0]
x = np.where(s == 0.4665162)
print(int(x[0]))

8888


In [56]:
sim_id = []
for s in sim:
    for i,cos in enumerate(s):
        sim_id.append((i,cos))


KeyboardInterrupt: 

In [55]:
sim_id[0]

(0,
 array([1.        , 0.18188246, 0.2831755 , ..., 0.3247849 , 0.15105638,
        0.2629717 ], dtype=float32))

In [31]:
recommendations

{}

In [10]:
ratings_train, ratings_test = train_test_split(ratings_small, test_size=0.2)


Before
80003
20001
After
41722
10444


In [ ]:
print('Before')
print(len(ratings_train))
print(len(ratings_test))

movieid_list = df['id'].to_list()
movieid_list = list(map(int, movieid_list))

print('After')
for r in ratings_train['movieId']:
    try:
        mid = r
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == int(mid)]
        index = movieid_list.index(int(tmdbId))
    except ValueError:
        ratings_train = ratings_train[ratings_train['movieId'] != mid]

print(len(ratings_train))

for r in ratings_test['movieId']:
    try:
        mid = r
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == int(mid)]
        index = movieid_list.index(int(tmdbId))
    except ValueError:
        ratings_test = ratings_test[ratings_test['movieId'] != mid]

print(len(ratings_test))


In [44]:
# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}

userid_list = ratings_train['userId'].unique()
movieid_list = df['id'].to_list()
movieid_list = list(map(int, movieid_list))

# get all movie vector rated from user
def get_user_rated_movies(ids):
        movie_vector_list = []
        for i in ids:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
            try:
                index = movieid_list.index(int(tmdbId))
                movie_vector = bert_embeddings[index]
                movie_vector_list.append(movie_vector)
            except ValueError:
                pass

        movie_vectors = movie_vector_list
        return movie_vectors


def create_user_vectors(uid):
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
        user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
        user_rated_movies_vector_array = np.array(user_rated_movies_vector_list)
        #user_vector = user_rated_movies_vector_array.sum(axis=0)
        user_vector = np.mean(user_rated_movies_vector_array, axis=0)
        all_user_profiles[uid] = user_vector


        
# Sorting Threading
recommendation_users = {}

def sort_user_vectors(uid):
    if uid in all_user_profiles.keys():
        user_vec = all_user_profiles[uid]
        cosine_similarity_user_movies = cosine_similarity(user_vec, bert_embeddings)
        top_similar = cosine_similarity_user_movies[0].argsort()[:100:-1]
        recommendation_users[uid] = [(cosine_similarity_user_movies[0][i], movies['id'][i]) for i in top_similar]

In [50]:
all_user_profiles = {}
uid = 2
user_profile = ratings_train.loc[ratings_train['userId'] == uid]
liste = user_profile['movieId'].to_list()
user_rated_movies_vector_list = get_user_rated_movies(liste[:2])
create_user_vectors(uid)
#len(all_user_profiles[uid])


In [51]:
tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == 485]
index = movieid_list.index(int(tmdbId))

In [55]:
len(user_profile)

26

In [56]:
all_user_profiles = {}
count_to_big = 0
count_false_int = 0
recommendation_users = {}

userid_list = ratings_train['userId'].unique()
movieid_list = df['id'].to_list()
# changes movieId type to int and deletes all false entrys

start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(create_user_vectors, userid_list)

end = timer()
print ("Time Calculate Vectors HH:MM:SS: ",timedelta(seconds=end-start))


start = timer()

with ThreadPoolExecutor(max_workers=8) as executer:
    executer.map(sort_user_vectors, userid_list)

end = timer()
print ("Time Sorting HH:MM:SS: ",timedelta(seconds=end-start))

C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Admin\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Admin\AppData\Local\P

KeyboardInterrupt: 

In [14]:
print(recommendation_users[1][:10])
print(recommendation_users[14][:10])
#m = movies.loc[movies['id'] == '385722']
#print(m['title'] + " " + m['overview'] )


KeyError: 1

In [11]:
%%time 
percentage_already_seen = []
for uid in userid_list:
    if uid in recommendation_users.keys():
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
        user_recommendation = recommendation_users[uid]
        
        user_profile_mid_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                print("ValueError: " + str(tmdbId))
        
        m_count = 0
        for mid, cos in user_recommendation[:len(user_profile)]:
            try:
                if int(mid) in user_tmdb_list:
                    m_count += 1
            except ValueError:
                print("ValueError: " + str(mid) + " not in tmdbid list")    

        if m_count == len(user_profile):
            print(str(uid) + " : the first movies are already viewed")
        else:
            print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

        p = m_count/len(user_profile)
        percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))

0 of 146 in first recommended already seen
0 of 151 in first recommended already seen
0 of 100 in first recommended already seen
0 of 562 in first recommended already seen
0 of 415 in first recommended already seen
0 of 757 in first recommended already seen
0 of 239 in first recommended already seen
0 of 233 in first recommended already seen
0 of 56 in first recommended already seen
0 of 222 in first recommended already seen
0 of 42 in first recommended already seen
0 of 71 in first recommended already seen
0 of 78 in first recommended already seen
0 of 156 in first recommended already seen
0 of 19 in first recommended already seen
0 of 82 in first recommended already seen
0 of 18 in first recommended already seen
0 of 153 in first recommended already seen
0 of 207 in first recommended already seen
0 of 110 in first recommended already seen
0 of 146 in first recommended already seen
0 of 61 in first recommended already seen
0 of 83 in first recommended already seen
0 of 81 in first rec

In [12]:
recommendation_users_top = recommendation_users
for k, v in recommendation_users_top.items():
    recommendation_users_top[k] = v[:10000]

len(recommendation_users_top[3])

10000

In [13]:
# Remove movies used to create User Vector

cleaned_recommendations = {}
for uid in userid_list:
    if uid in all_user_profiles.keys():
        user_profile = ratings_train.loc[ratings_train['userId'] == uid]
        user_recommendation = recommendation_users_top[uid]

        user_profile_mid_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                count_false_int = count_false_int + 1
            
        m_count = 0
        for cos, mid in user_recommendation:
            try:
                if int(mid) in user_tmdb_list:
                    m_count += 1
                    user_recommendation.remove((cos,mid))
            except ValueError:
                #print("ValueError: " + str(mid) + " not in tmdbid list")    #Fehler weil, schon gesucht aber nicht gefunden // skipped because to big sample size
                pass

        cleaned_recommendations[uid] = user_recommendation

In [14]:
# Calculate Percentage of user rated movies from test split in recommended, top 10/50/100
top_value = 100
user_percentages = []
for uid in userid_list:
    user_profile = ratings_test.loc[ratings_test['userId'] == uid]
    found_movies = 0
    if uid in recommendation_users.keys():
        user_recommendations = cleaned_recommendations[uid]
        #print(len(user_recommendations))
        #tmdbId_list = user_profile['tmdbId'].to_list():

        user_profile_mid_list = user_profile['movieId'].to_list()
        user_tmdb_list = []
        for m in user_profile_mid_list:
            tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
            try:
                user_tmdb_list.append(int(tmdbId))
            except ValueError:
                #print("ValueError: " + str(tmdbId))
                pass
        for cos, mid in user_recommendations[:top_value]:
            try:
                if int(mid) in user_tmdb_list:
                    found_movies = found_movies + 1
            except ValueError:
                #print("ValueError: " + str(mid) + " not in tmdbid list")
                pass
       
    user_percentages.append(found_movies/len(user_profile))

mean_percentage = sum(user_percentages)/len(user_percentages)
print("Mean Percentage of good Recommendations (based on Test Split): " + str(mean_percentage))

Mean Percentage of good Recommendations (based on Test Split): 0.005468930474354847


In [ ]:
# distilbert-base-nil-stsb
Mean Percentage of good Recommendations (based on Test Split): 0.02751855370502398
# paraphrase-distilroberta-base-v1
Mean Percentage of good Recommendations (based on Test Split): 0.0019663851631064747
# stsb-roberta-large
Mean Percentage of good Recommendations (based on Test Split): 0.005468930474354847
